In [1]:
import numpy as np
import math
import scipy.stats as st
import matplotlib.pyplot as plt
import time

In [2]:
def select_b_and_s_b(mean):
    b = np.argsort(mean)[0]
    s_b = np.argsort(mean)[1]
    return b, s_b

def Initialize_n_0_sim(k, T, action, alpha = 0.9, base_policy = [0, 1], n = 5):
    sample = np.zeros((k, n))
    for j in range(k):
        cost = np.zeros(n)
        for l in range(n):
            for i in range(T):
                if i == 0:
                    next_state = np.random.choice([1,2], p = action[j])
                    cost[l] += 1
                    state = next_state
                else:
                    if state == 1:
                        next_state = np.random.choice([1,2], p = base_policy)
                        cost[l] += alpha ** i
                        state = next_state
                    else:
                        next_state = np.random.choice([1,2], p = [0.5, 0.5])
                        state = next_state
            sample[j][l] = cost[l]
    return sample

def addistional_sim(T, action,n,  alpha = 0.9, base_policy = [0, 1], state = 1):
    sample = np.zeros(n)
    for l in range(n):
        cost = 0
        for i in range(T):
            if i == 0:
                next_state = np.random.choice([1,2], p = action)
                cost += 1
                state = next_state
            else:
                if state == 1:
                    next_state = np.random.choice([1,2], p = base_policy)
                    cost += alpha ** i
                    state = next_state
                else:
                    next_state = np.random.choice([1,2], p = [0.5, 0.5])
                    state = next_state
        sample[l] = cost
    
    return sample

def n_0_estimate(sample):
    mean_initial, var_initial = np.zeros(len(sample)), np.zeros(len(sample))
    # Generate new mean and new variance according to the simulation outputs(i.e., random_sample)
    for i in range(len(sample)):
        mean_initial[i] = np.mean(sample[i])
        var_initial[i] = np.var(sample[i], ddof = 1)
    return mean_initial, var_initial

def initial_ratio(k, mean, var):
    best_index = select_b_and_s_b(mean)[0]
    second_index = select_b_and_s_b(mean)[1]
    N = np.zeros(k)
    N[second_index] = 1
    
    for i in range(k):
        if i != best_index and i!= second_index:
            temp = (mean[best_index] - mean[second_index]) / (mean[best_index] - mean[i])
            N[i] = temp ** 2 * var[i] / var[second_index]
    sum_ = 0
    for j in range(k):
        if j != best_index:
            sum_ += (N[j]) ** 2 / var[j]
    N[best_index] = math.sqrt(var[best_index] * sum_)
    ratio_sum = N.sum()
    prop = np.zeros(k)
    for i in range(0, k):
        prop[i] = N[i] / ratio_sum
    return prop

def allocation(prob, tri_del, k):
    al_sim, I, D = np.zeros(k), np.zeros(k), np.zeros(k)
    for i in range(k):
        al_sim[i] = tri_del * prob[i] 
    # Find the integer part and decimal part of N
    for i in range(k):
        I[i] = math.modf(al_sim[i])[1]
        D[i] = math.modf(al_sim[i])[0]
    # Calculate the remaining computing budget 
    remain = tri_del - I.sum()
    # Return the sorted index of decimal part
    d = np.argsort(-D)
    i = 0
    # Allocata the remaining budget to the designs which have higher decimal parts
    while(remain):
        I[d[i]] += 1
        i += 1
        remain -= 1
    return I

def OCBAPI(k = 20, bp = [0, 1], alpha = 0.9, e = 0.01, F = 1, n_0 = 20, tri_del = 100, total_budget = 10000):
    actual_best = 0
    select_best= [ ]
    state = 1
    c = e / 2
    T = math.ceil(math.log((c * (1 - alpha)) / F) / math.log(alpha))
    action = np.zeros((k, state + 1))
    for i in range(k):
        action[i] = [i/k, 1- (i/k)]
    count_best = np.zeros(int((total_budget - k * n_0) / (tri_del)))
    n_0_samples = Initialize_n_0_sim(k, T, action, alpha, bp, n_0)
    mean_init, var_init = n_0_estimate(n_0_samples)
    n_0_sims = np.full(k, n_0)
    additional_sims = np.zeros(k)
    sample_store = [[]] * k
    for i in range(k):
        sample_store[i] = []
        sample_store[i].extend(list(n_0_samples[i]))
      
    current_budget = n_0 * k
    while(current_budget < total_budget):
        current_budget += tri_del
        probs = initial_ratio(k, mean_init, var_init)
        allo = np.array(allocation(probs, current_budget , k), dtype = np.int)
        for i in range(k):
            additional_sims[i] = np.maximum(allo[i] - n_0_sims[i], 0)
        
        to_do = np.where(additional_sims > 0)
        for i in to_do[0]:
            
            new_sample = addistional_sim(T, action[i], int(additional_sims[i]), alpha, bp, state = 1)
            sample_store[i].extend(list(new_sample))
            mean_init[i] = np.mean(sample_store[i])
            var_init[i] = np.var(sample_store[i], ddof = 1)
            n_0_sims[i] = allo[i]

        select_best.append(select_b_and_s_b(mean_init)[0])
    for i in range(int((total_budget - k * n_0) / (tri_del))):
        if select_best[i] == actual_best:
            count_best[i] = 1
    return count_best

In [5]:
OCBAPI(k = 20,  bp = [0.95, 0.05], alpha = 0.9, e = 0.01, F = 1, n_0 = 20, tri_del = 100, total_budget = 13600)

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [21]:
start = time.process_time()
test_num = 10
k = 20
tem = np.zeros(int((total_budget - k * n_0) / (tri_del)))
PCS = np.zeros(int((total_budget - k * n_0) / (tri_del)))
for i in range(test_num):
    tem = OCBAPI(k,  bp = [0.95, 0.05], alpha = 0.9, e = 0.01, F = 1, n_0 = 20, tri_del = 100, total_budget = 13600)
    PCS += tem


end = time.process_time()
print(PCS)
print('Running time is %.3f seconds' %(end - start))

[ 5.  5.  6.  7.  9.  9.  8.  9.  9.  9. 10. 10. 10.  9.  9. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10.]
Running time is 172.094 seconds


In [18]:
start = time.process_time()
test_num = 200
k = 20
n_0 = 20
tri_del = 100
total_budget = 13600
T = 73
tem = np.zeros(int((total_budget - k * n_0) / (tri_del)))
PCS = np.zeros(int((total_budget - k * n_0) / (tri_del)))
for i in range(test_num):
    tem = OCBAPI(k,  bp = [0.95, 0.05], alpha = 0.9, e = 0.01, F = 1, n_0 = 20, tri_del = 100, total_budget = 13600)
    PCS += tem


end = time.process_time()
print(PCS)
print('Running time is %.3f seconds' %(end - start))

[104. 119. 138. 153. 159. 156. 161. 160. 163. 166. 169. 170. 173. 172.
 172. 172. 176. 176. 180. 179. 184. 185. 185. 185. 187. 189. 187. 187.
 187. 187. 187. 189. 189. 190. 190. 190. 190. 190. 190. 191. 190. 192.
 192. 193. 193. 194. 194. 194. 195. 195. 195. 195. 195. 196. 197. 197.
 197. 198. 199. 199. 199. 199. 199. 199. 199. 199. 199. 199. 199. 199.
 199. 199. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200.]
Running time is 3246.844 seconds


In [22]:
start = time.process_time()
test_num = 500
k = 20
tem = np.zeros(int((total_budget - k * n_0) / (tri_del)))
PCS = np.zeros(int((total_budget - k * n_0) / (tri_del)))
for i in range(test_num):
    tem = OCBAPI(k,  bp = [0.95, 0.05], alpha = 0.9, e = 0.01, F = 1, n_0 = 20, tri_del = 100, total_budget = 13600)
    PCS += tem


end = time.process_time()
print(PCS)
print('Running time is %.3f seconds' %(end - start))

[261. 330. 353. 371. 382. 382. 396. 405. 405. 412. 419. 431. 429. 434.
 437. 441. 442. 441. 445. 451. 451. 454. 457. 456. 458. 460. 462. 461.
 464. 466. 465. 467. 469. 471. 471. 475. 475. 478. 479. 481. 481. 480.
 481. 480. 480. 483. 483. 484. 486. 486. 486. 487. 486. 489. 489. 489.
 489. 489. 491. 491. 492. 492. 493. 494. 494. 494. 494. 494. 494. 495.
 495. 495. 495. 495. 495. 495. 495. 495. 495. 495. 495. 495. 495. 495.
 496. 496. 496. 496. 496. 496. 496. 497. 497. 497. 497. 497. 497. 497.
 497. 497. 497. 497. 498. 498. 498. 498. 498. 498. 498. 498. 498. 498.
 499. 499. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
 500. 500. 500. 500. 500. 500.]
Running time is 9365.969 seconds
